# Introduction to azure.ai.ml:
azure.ai.ml <- this is version 2 of the SDK

# Connect to the workspace

## Prerequisite #1 : your info

in V2, you use the MLClient to connect to your workspace. This can be found in two places: 

1. The azure portal when you look at the overview in the [AML resource](https://portal.azure.com/#@MngEnv319812.onmicrosoft.com/resource/subscriptions/40fd749f-c876-4e1e-87c0-835314a59e42/resourceGroups/dp100-main-demo/providers/Microsoft.MachineLearningServices/workspaces/training-ml104/overview). 

2. The top right corner of the AML directory

From this link, you can extract:
- subscripton Id
- resource group
- workspace

## Prerequisite #2: your permissions/credentials

We are using [DefaultAzureCredential](https://learn.microsoft.com/en-us/python/api/azure-identity/azure.identity.defaultazurecredential?view=azure-python) to get access to workspace.

DefaultAzureCredential should be capable of handling most Azure SDK authentication scenarios. Reference [here](https://learn.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python) for all available credentials if it does not work for you.



In [ ]:
client_config = {
        "resource_group": "",#Place resource group here,
        "subscription_id":"", #Place subscription ID here,
        "workspace_name": "", #Place AML workspace name here,
    }

In [7]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# enter details of your AML workspace


# get a handle to the workspace you can also just call the variable mlclient
ws = MLClient(
    credential = DefaultAzureCredential(), 
    resource_group_name= client_config["resource_group"], #resource_group
    subscription_id= client_config["subscription_id"], #AML subscription id
    workspace_name= client_config["workspace_name"] # workspace
)



In [1]:
for ws in ws.workspaces.list():
    print(ws)

NameError: name 'ws' is not defined

# What does my MLClient(ws) grant me access to do (via code):

1. Run Standalone Jobs - run a discrete ML activity as Job. This job can be run locally or on the cloud. Types of jobs, we can organise using the sdk:
- Command - run a command (Python, R, Windows Command, Linux Shell etc.)
- Sweep - run a hyperparameter sweep on your Command

2. Run multiple jobs using our improved Pipelines
- Run a series of commands stitched into a pipeline
- Components - run pipelines using reusable components

3. Making predictions/inferencing at batch/ singular

4. **Manage AML resources and assets – workspace, compute, datastores, datasets, environments, models **


[documentation](https://learn.microsoft.com/en-us/python/api/overview/azure/ai-ml-readme?view=azure-python)

# What does the MLClient have to help me inside of my workspace

begin_create_or_update: Creates or updates an Azure ML resource asynchronously.

create_or_update: Creates or updates an Azure ML resource.

from_config: 
- Return a workspace object from an existing Azure Machine Learning Workspace.

- Reads workspace configuration from a file. Throws an exception if the config file can't be found.

- The method provides a simple way to reuse the same workspace across multiple Python notebooks or projects. Users can save the workspace Azure Resource Manager (ARM) properties using the workspace.write_config method, and use this method to load the same workspace in different Python notebooks or projects without retyping the workspace ARM properties.

In [4]:
# Example of retrieving from_config

try:
    ws = MLClient.from_config(credential=DefaultAzureCredential())
except Exception as ex:
    # write and reload from config file
    import json, os

    config_path = "../.azureml/config.json"
    os.makedirs(os.path.dirname(config_path), exist_ok=True)
    with open(config_path, "w") as fo:
        fo.write(json.dumps(client_config))
    ws = MLClient.from_config(credential=DefaultAzureCredential(), path=config_path)
print(ws)

NameError: name 'client_config' is not defined

# Lets create a compute resource through code

In [14]:
for compute in ws.compute.list():
    if compute.provisioning_state != "Failed":
        print(f"{compute.type}: {compute.name} :{compute.size}")

computeinstance: LittleVMBoiTheSecond :STANDARD_DS11_V2
amlcompute: ThreeTimesMyVMBoi :STANDARD_DS3_V2
amlcompute: CantClustThis :STANDARD_D2_V2
amlcompute: TwoMachinedCluster :STANDARD_DS12_V2
amlcompute: demo-cluster :STANDARD_DS11_V2
amlcompute: your-compute-cluster :STANDARD_DS11_V2


In [2]:
ws.compute.get("CantClustThis")

NameError: name 'ws' is not defined

In [3]:
from azure.ai.ml.entities import AmlCompute, ComputeInstance

# specify aml compute name.
cpu_compute_name = "singleComputeMachine"

try:
    ws.compute.get(cpu_compute_name)
except Exception:
    print("Creating a new cpu compute target...")
    compute = ComputeInstance(
        name=cpu_compute_name, vm_size="STANDARD_DS11_V2"
    )
    ws.compute.begin_create_or_update(compute)

ModuleNotFoundError: No module named 'azure'

1.5 Create a workspace for use with Azure Private Link
When using private link, your workspace cannot use the regular Azure Container Registry tasks compute for image building. Hence, you must set the image_build_compute property to some other CPU compute cluster name to use for Docker image environment building. You can also specify whether the private link workspace should be accessible over the internet using the public_network_access property. After workspace creation, create a private link endpoint for your workspace. For more details, see [Secure Azure Machine Learning workspace resources using virtual networks (VNets)](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-secure-workspace-vnet?tabs=pe%2Ccli).


# What about finding data?

In [65]:
for workspace in ws.datastores.list():
    print(workspace.name)

v2_registerblob_test
snowman_2022_12_13
coffee_2022_11_08
blobstorage_2022_11_08
azureml_globaldatasets
blobstorage_2022_11_02
dp100_adls_model
dp100_adls_bronze
workspaceworkingdirectory
workspaceartifactstore
workspacefilestore
workspaceblobstore
